# Test WaffleIron Model

In [1]:
from models.waffleiron.segmenter import Segmenter

Using torch.scatter_reduce for 3D to 2D projection.
Using torch.scatter_reduce for 3D to 2D projection.


In [2]:
model = Segmenter(
    input_channels=5,
    feat_channels=768,
    depth=48,
    grid_shape=[[256, 256], [256, 32], [256, 32]],
    nb_class=16,
    #drop_path_prob=config["waffleiron"]["drop_path"],
    layer_norm=True,
)

In [3]:
import torch

# Load pretrained model
ckpt = torch.load('./saved_models/ckpt_last_scalr.pth', map_location="cuda:0")
ckpt = ckpt["net"]

In [4]:
print(ckpt.keys())

dict_keys(['module.embed.norm.weight', 'module.embed.norm.bias', 'module.embed.norm.running_mean', 'module.embed.norm.running_var', 'module.embed.norm.num_batches_tracked', 'module.embed.conv1.weight', 'module.embed.conv1.bias', 'module.embed.conv2.0.weight', 'module.embed.conv2.0.bias', 'module.embed.conv2.0.running_mean', 'module.embed.conv2.0.running_var', 'module.embed.conv2.0.num_batches_tracked', 'module.embed.conv2.1.weight', 'module.embed.conv2.2.weight', 'module.embed.conv2.2.bias', 'module.embed.conv2.2.running_mean', 'module.embed.conv2.2.running_var', 'module.embed.conv2.2.num_batches_tracked', 'module.embed.conv2.4.weight', 'module.embed.final.weight', 'module.embed.final.bias', 'module.waffleiron.channel_mix.0.scale.weight', 'module.waffleiron.channel_mix.0.norm.weight', 'module.waffleiron.channel_mix.0.norm.bias', 'module.waffleiron.channel_mix.0.mlp.0.weight', 'module.waffleiron.channel_mix.0.mlp.0.bias', 'module.waffleiron.channel_mix.0.mlp.2.weight', 'module.waffleiro

In [5]:
new_ckpt = {}
for k in ckpt.keys():
    if k.startswith("module"):
        if k.startswith("module.classif.0"):
            continue
        elif k.startswith("module.classif.1"):
            new_ckpt["classif" + k[len("module.classif.1") :]] = ckpt[k]
        else:
            new_ckpt[k[len("module.") :]] = ckpt[k]
    else:
        new_ckpt[k] = ckpt[k]

In [6]:
new_ckpt.get("classif.weight").shape

torch.Size([16, 768, 1])

In [7]:
model.load_state_dict(new_ckpt)

<All keys matched successfully>

## Model loaded --> Test features somehow

In [ ]:
import os
import torch
from auxiliary.process_data.nuscenes.nuscenes_dataset import DatasetTrainVal

print("Creating dataloader...", flush=True)

target = '/root/main/dataset/processed'

filelist_train = [os.path.join(target, 'train_pointclouds', fname) for fname in os.listdir(os.path.join(target, 'train_pointclouds')) if os.path.splitext(fname)[1]==".npy"]
filelist_train.sort()
filelist_val = filelist_train[:3]
filelist_train = filelist_train[3:]

ds = DatasetTrainVal(filelist_train, os.path.join(target, 'train_pointclouds'),
                            training=True,
                            npoints=2000,
                            iteration_number=(2*10)*10,
                            jitter=0.2)
train_loader = torch.utils.data.DataLoader(ds, batch_size=1, shuffle=False, # Change batch_size
                                    num_workers=1)

In [ ]:
for t in train_loader:
    print(torch.bincount(t['target'][0]))
    x = input()

In [8]:
from plyfile import PlyData, PlyElement
import numpy as np
import os

train_filenames = ["Lille1_1.ply",  "Lille1_2.ply",  "Lille2.ply",  "Paris.ply",]
destdir = '/root/main/dataset/processed/'
rootdir = '/root/main/dataset/'

filenames = train_filenames
save_dir = os.path.join(destdir,"train_pointclouds")
pts_all = {}
for filename in filenames:
    fname = os.path.join(rootdir, "training_10_classes", filename)
    plydata = PlyData.read(fname)
    x = plydata["vertex"].data["x"].astype(np.float32)
    y = plydata["vertex"].data["y"].astype(np.float32)
    z = plydata["vertex"].data["z"].astype(np.float32)
    reflectance = plydata["vertex"].data["reflectance"].astype(np.float32)
    label = plydata["vertex"].data["class"].astype(np.float32)
    pts = np.concatenate([
        np.expand_dims(x,1),
        np.expand_dims(y,1),
        np.expand_dims(z,1),
        np.expand_dims(reflectance,1),
        np.expand_dims(label,1),
    ], axis=1).astype(np.float32)
    pts_all[filename] = pts

In [9]:
pts_all["Lille1_1.ply"][:,1]

array([  83.03686 ,   83.05253 ,   83.117584, ..., -386.54715 ,
       -386.5629  , -386.97958 ], dtype=float32)

In [10]:
print(min(pts_all["Lille1_1.ply"][:,0]))
print(max(pts_all["Lille1_1.ply"][:,0]))
print(min(pts_all["Lille1_1.ply"][:,1]))
print(max(pts_all["Lille1_1.ply"][:,1]))
print(min(pts_all["Lille1_1.ply"][:,2]))
print(max(pts_all["Lille1_1.ply"][:,2]))

-360.1785
67.185555
-412.13937
101.893524
32.78496
66.74454


In [11]:
(101.893524 - (-412.13937))/20 

25.701644699999996

In [12]:
20 * 25 * 3

1500

In [13]:
print(np.mean(pts_all["Lille1_1.ply"][:,3]))

18.705505


In [14]:
print(np.std(pts_all["Lille1_1.ply"][:,3]))

23.756725


In [ ]:
import torch

pc_here = torch.from_numpy(pts_all["Lille1_1.ply"]).cuda() 

In [ ]:
def voxelize_pointcloud(points, voxel_size):
    """
    Voxelizes a point cloud.

    Args:
        points (np.ndarray): The point cloud as an Nx3 array of XYZ coordinates.
        voxel_size (float): The size of the voxels (uniform along all axes).
        
    Returns:
        voxel_indices (np.ndarray): The voxel indices for each point.
        voxel_grid (dict): A dictionary where keys are voxel indices (tuples) and values are points in that voxel.
    """
    # Normalize points by voxel size
    voxel_indices = np.floor(points[:,:3] / voxel_size).astype(np.int32)
    
    # Create a dictionary to store points in each voxel
    voxel_grid = {}
    
    for idx, voxel in tqdm(enumerate(voxel_indices)):
        voxel_key = tuple(voxel)  # Use tuple to make it hashable for the dictionary
        if voxel_key not in voxel_grid:
            voxel_grid[voxel_key] = []
        voxel_grid[voxel_key].append(points[idx])
    
    # Convert lists to arrays
    for key in voxel_grid:
        voxel_grid[key] = np.array(voxel_grid[key])
    
    return voxel_indices, voxel_grid

In [ ]:
from tqdm import tqdm
#point_cloud = np.random.rand(1000, 3) * 100  # 1000 points in a 100x100x100 space
voxel_size = 20.0  # Voxel size of 5 units

# Voxelize the point cloud
voxel_indices, voxel_grid = voxelize_pointcloud(pts_all["Lille1_1.ply"], voxel_size)

In [ ]:
# Print some results
print("Voxel indices:\n", voxel_indices[900])  # Show first 10 voxel indices
print("\nNumber of unique voxels:", len(voxel_grid))

In [ ]:
for i in list(voxel_grid.keys()):
    print("\nPoints in a specific voxel:", len(voxel_grid[i]))
    if len(voxel_grid[i]) < 100:
        voxel_grid.pop(i)

In [ ]:
for i in list(voxel_grid.keys()):
    print("\nPoints in a specific voxel:", len(voxel_grid[i]))
    voxel_grid[i][:,4] = voxel_grid[i][:,4] - 1

In [ ]:
for i in list(voxel_grid.keys()):
    print(voxel_grid[i][:,4])

In [15]:
torch.cuda.set_device("cuda:0")
model = model.cuda("cuda:0")

In [16]:
model.eval()

Segmenter(
  (embed): Embedding(
    (norm): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1): Conv1d(5, 768, kernel_size=(1,), stride=(1,))
    (conv2): Sequential(
      (0): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv2d(5, 768, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (2): BatchNorm2d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
      (4): Conv2d(768, 768, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (final): Conv1d(1536, 768, kernel_size=(1,), stride=(1,))
  )
  (waffleiron): WaffleIron(
    (channel_mix): ModuleList(
      (0): ChannelMix(
        (norm): myLayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Conv1d(768, 768, kernel_size=(1,), stride=(1,))
          (1): ReLU(inplace=True)
          (2): Conv1d(768, 768, kernel_size=(1,), stride=(1,))
        )
        (

In [17]:
import torch
import numpy as np
import utils.transforms as tr
from torch.utils.data import Dataset
from scipy.spatial import cKDTree as KDTree

class PCDataset(Dataset):
    def __init__(
        self,
        rootdir=None,
        phase="train",
        input_feat="intensity",
        voxel_size=0.1,
        train_augmentations=None,
        dim_proj=[
            0,
        ],
        grids_shape=[(256, 256)],
        fov_xyz=(
            (-1.0, -1.0, -1.0),
            (1.0, 1.0, 1.0),
        ),
        num_neighbors=16,
    ):
        super().__init__()

        # Dataset split
        self.phase = phase
        assert self.phase in ["train", "val", "trainval", "test"]

        # Root directory of dataset
        self.rootdir = rootdir

        # Input features to compute for each point
        self.input_feat = input_feat

        # Downsample input point cloud by small voxelization
        self.downsample = tr.Voxelize(
            dims=(0, 1, 2),
            voxel_size=voxel_size,
            random=(self.phase == "train" or self.phase == "trainval"),
        )

        # Field of view
        assert len(fov_xyz[0]) == len(
            fov_xyz[1]
        ), "Min and Max FOV must have the same length."
        for i, (min, max) in enumerate(zip(*fov_xyz)):
            assert (
                min < max
            ), f"Field of view: min ({min}) < max ({max}) is expected on dimension {i}."
        self.fov_xyz = np.concatenate([np.array(f)[None] for f in fov_xyz], axis=0)
        self.crop_to_fov = tr.Crop(dims=(0, 1, 2), fov=fov_xyz)

        # Grid shape for projection in 2D
        assert len(grids_shape) == len(dim_proj)
        self.dim_proj = dim_proj
        self.grids_shape = [np.array(g) for g in grids_shape]
        self.lut_axis_plane = {0: (1, 2), 1: (0, 2), 2: (0, 1)}

        # Number of neighbors for embedding layer
        assert num_neighbors > 0
        self.num_neighbors = num_neighbors

        # Train time augmentations
        if train_augmentations is not None:
            assert self.phase in ["train", "trainval"]
        self.train_augmentations = train_augmentations
        
        self.list_frames = ["Lille1_1.ply"] # ,  "Lille1_2.ply",  "Lille2.ply",  "Paris.ply",
        self.mean_int = 18.705505
        self.std_int = 23.756725

    def get_occupied_2d_cells(self, pc):
        """Return mapping between 3D point and corresponding 2D cell"""
        cell_ind = []
        for dim, grid in zip(self.dim_proj, self.grids_shape):
            # Get plane of which to project
            dims = self.lut_axis_plane[dim]
            # Compute grid resolution
            res = (self.fov_xyz[1, dims] - self.fov_xyz[0, dims]) / grid[None]
            # Shift and quantize point cloud
            pc_quant = ((pc[:, dims] - self.fov_xyz[0, dims]) / res).astype("int")
            # Check that the point cloud fits on the grid
            min, max = pc_quant.min(0), pc_quant.max(0)
            assert min[0] >= 0 and min[1] >= 0, print(
                "Some points are outside the FOV:", pc[:, :3].min(0), self.fov_xyz
            )
            assert max[0] < grid[0] and max[1] < grid[1], print(
                "Some points are outside the FOV:", pc[:, :3].min(0), self.fov_xyz
            )
            # Transform quantized coordinates to cell indices for projection on 2D plane
            temp = pc_quant[:, 0] * grid[1] + pc_quant[:, 1]
            cell_ind.append(temp[None])
        return np.vstack(cell_ind)

    def prepare_input_features(self, pc_orig):
        # Concatenate desired input features to coordinates
        pc = [pc_orig[:, :3]]  # Initialize with coordinates
        for type in self.input_feat:
            if type == "intensity":
                intensity = pc_orig[:, 3:]
                intensity = (intensity - self.mean_int) / self.std_int
                pc.append(intensity)
            elif type == "height":
                pc.append(pc_orig[:, 2:3])
            elif type == "radius":
                r_xyz = np.linalg.norm(pc_orig[:, :3], axis=1, keepdims=True)
                pc.append(r_xyz)
            elif type == "xyz":
                xyz = pc_orig[:, :3]
                pc.append(xyz)
            elif type == "constant":
                pc.append(np.ones((pc_orig.shape[0], 1)))
            else:
                raise ValueError(f"Unknown feature: {type}")
        return np.concatenate(pc, 1)

    def load_pc(self, index):
        fname = os.path.join(self.rootdir, "training_10_classes", self.list_frames[index])
        plydata = PlyData.read(fname)
        x = plydata["vertex"].data["x"].astype(np.float32)
        y = plydata["vertex"].data["y"].astype(np.float32)
        z = plydata["vertex"].data["z"].astype(np.float32)
        reflectance = plydata["vertex"].data["reflectance"].astype(np.float32)
        label = plydata["vertex"].data["class"].astype(np.float32)
        label = label-1
        pts = np.concatenate([
            np.expand_dims(x,1),
            np.expand_dims(y,1),
            np.expand_dims(z,1),
            np.expand_dims(reflectance,1)
        ], axis=1).astype(np.float32)
        
        return pts, np.expand_dims(label,1), self.list_frames[index]

    def __len__(self):
        return len(self.list_frames)

    def __getitem__(self, index):
        # Load original point cloud
        pc_orig, labels_orig, filename = self.load_pc(index)

        # Prepare input feature
        pc_orig = self.prepare_input_features(pc_orig)

        # Voxelization
        pc, labels = self.downsample(pc_orig, labels_orig)

        # Augment data
        if self.train_augmentations is not None:
            pc, labels = self.train_augmentations(pc, labels)

        # Crop to fov
        pc, labels = self.crop_to_fov(pc, labels)

        # For each point, get index of corresponding 2D cells on projected grid
        cell_ind = self.get_occupied_2d_cells(pc)

        # Get neighbors for point embedding layer providing tokens to waffleiron backbone
        kdtree = KDTree(pc[:, :3])
        assert pc.shape[0] > self.num_neighbors
        dist, neighbors_emb = kdtree.query(pc[:, :3], k=self.num_neighbors + 1)

        # Nearest neighbor interpolation to undo cropping & voxelisation at validation time
        if self.phase in ["train", "trainval"]:
            upsample = np.arange(pc.shape[0])
        else:
            _, upsample = kdtree.query(pc_orig[:, :3], k=1)

        # Output to return
        out = (
            # Point features
            pc[:, 3:].T[None],
            # Point labels of original entire point cloud
            labels if self.phase in ["train", "trainval"] else labels_orig,
            # Projection 2D -> 3D: index of 2D cells for each point
            cell_ind[None],
            # Neighborhood for point embedding layer, which provides tokens to waffleiron backbone
            neighbors_emb.T[None],
            # For interpolation from voxelized & cropped point cloud to original point cloud
            upsample,
            # Filename of original point cloud
            filename,
        )

        return out

In [18]:
class Collate:
    def __init__(self, num_points=None):
        self.num_points = num_points
        assert num_points is None or num_points > 0

    def __call__(self, list_data):

        # Extract all data
        list_of_data = (list(data) for data in zip(*list_data))
        feat, label_orig, cell_ind, neighbors_emb, upsample, filename = list_of_data

        # Zero-pad point clouds
        Nmax = np.max([f.shape[-1] for f in feat])
        if self.num_points is not None:
            assert Nmax <= self.num_points
        occupied_cells = []
        for i in range(len(feat)):
            feat[i], neighbors_emb[i], cell_ind[i], temp = zero_pad(
                feat[i],
                neighbors_emb[i],
                cell_ind[i],
                Nmax if self.num_points is None else self.num_points,
            )
            occupied_cells.append(temp)

        # Concatenate along batch dimension
        feat = torch.from_numpy(np.vstack(feat)).float()  # B x C x Nmax
        neighbors_emb = torch.from_numpy(np.vstack(neighbors_emb)).long()  # B x Nmax
        cell_ind = torch.from_numpy(
            np.vstack(cell_ind)
        ).long()  # B x nb_2d_cells x Nmax
        occupied_cells = torch.from_numpy(np.vstack(occupied_cells)).float()  # B x Nmax
        labels_orig = torch.from_numpy(np.hstack(label_orig)).long()
        upsample = [torch.from_numpy(u) for u in upsample]

        # Prepare output variables
        out = {
            "feat": feat,
            "neighbors_emb": neighbors_emb,
            "upsample": upsample,
            "labels_orig": labels_orig,
            "cell_ind": cell_ind,
            "occupied_cells": occupied_cells,
            "filename": filename,
        }

        return out


In [19]:
kwargs = {
        "rootdir": '/root/main/dataset/',
        "input_feat": ["xyz", "intensity"],
        "voxel_size": 5,
        "num_neighbors": 16,
        "dim_proj": [2, 1, 0],
        "grids_shape": [[256, 256], [256, 32], [256, 32]],
        "fov_xyz": [[-64, -64, -8], [64, 64, 8]],
    }

train_dataset = PCDataset(
        phase="train",
        **kwargs,
    )

In [20]:
train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=1,
        pin_memory=True,
        drop_last=True,
        collate_fn=Collate(),
    )

In [ ]:
for it, batch in enumerate(train_loader):
    
    if it == 0:

        # Network inputs
        feat = batch["feat"].cuda(0, non_blocking=True)
        labels = batch["labels_orig"].cuda(0, non_blocking=True)
        batch["upsample"] = [
            up.cuda(0, non_blocking=True) for up in batch["upsample"]
        ]
        cell_ind = batch["cell_ind"].cuda(0, non_blocking=True)
        occupied_cell = batch["occupied_cells"].cuda(0, non_blocking=True)
        neighbors_emb = batch["neighbors_emb"].cuda(0, non_blocking=True)
        net_inputs = (feat, cell_ind, occupied_cell, neighbors_emb)

        with torch.no_grad():
            out = model(*net_inputs)

        print(out)
    